In [1]:
import os
import json

from ipyleaflet import Map, SearchControl, Marker, AwesomeIcon, GeoJSON, LayerGroup

center = [19.1646, 72.8493]
m = Map(center=center, zoom=3)

In [2]:
marker = Marker(
    icon=AwesomeIcon(name="check", marker_color="green", icon_color="darkgreen")
)

In [3]:
# Search using url.

search = SearchControl(
    position="topleft",
    url="https://nominatim.openstreetmap.org/search?format=json&q={s}",
    zoom=5,
    property_name="display_name",
    marker=marker,
)
m.add(search)
m

Map(center=[19.1646, 72.8493], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [ ]:
marker.icon = AwesomeIcon(name="check", marker_color="red", icon_color="darkred")

In [ ]:
# Search in a GeoJSON layer

with open("countries.geo.json", "r") as f:
    data = json.load(f)

countries = GeoJSON(data=data)

m = Map(center=(19.1646, 72.8493), zoom=3)

search = SearchControl(
    position="topleft",
    layer=LayerGroup(layers=(countries,)),
    zoom=4,
    property_name="name",
    marker=marker,
    found_style={
        "opacity": 1,
        "dashArray": "9",
        "fillOpacity": 0.1,
        "weight": 1,
        "fillColor": "#FF0000",
        "color": "#FF6347",
    },
)
m.add(search)
m

In [ ]:
# Search in a GeoJSON layer with callbacks for searched feature.

with open("europe_110.geo.json") as f:
    data = json.load(f)

europe = GeoJSON(data=data)

m = Map(center=[61.9389, 69.7851], zoom=2)

search = SearchControl(
    position="topleft",
    layer=LayerGroup(layers=(europe,)),
    property_name="name",
)
m.add(search)
m

In [ ]:
from ipywidgets import HTML
from ipyleaflet import Popup
from shapely.geometry import shape


def add_popup(feature, **kwargs):
    for layer in m.layers:
        if isinstance(layer, Popup):
            m.remove_layer(layer)
    p = shape(feature["geometry"])
    center = p.centroid.y, p.centroid.x
    properties = feature["properties"]
    message = HTML()
    message.value = f"""<h4>EU population density</h4><h4>
                        <b>{properties["name"]}</b>
                        </h4>{properties["pop_est"]} people / mi^2
                     """
    popup = Popup(
        location=center,
        child=message,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False,
    )

    m.add(popup)


search.on_feature_found(add_popup)